In [1]:
#import relevant packages
import pandas as pd
import numpy as np
import pickle as pk
import pycountry_convert as pc

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances

pd.options.display.max_colwidth = 500

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\junaid.shaikh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
'''
#INFERENCE parameters 
req_country = 'aus'
req_lang = 'eng'
req_study_type = 'B2B'
req_subject = 'other'

req_continent = pc.convert_continent_code_to_continent_name(pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(req_country.upper())))
'''

In [2]:
#INFERENCE parameters
DATAPATH = 'data/final_transformed_data.csv'
ir_query = pd.read_csv(DATAPATH, nrows=1)[['continent', 'sample_pulls__language', 'projects__study_types_ids', 'projects__study_types_subject_ids','continent_language_study_combine','projects__target_groups_qualifications_combine']]
req_continent_language_study = ir_query.continent_language_study_combine.values.tolist()[0]
req_tg_qualification = ir_query.projects__target_groups_qualifications_combine.values.tolist()[0]
print(req_continent_language_study)
print(req_tg_qualification)


Europe eng Healthcare
{'AGE': ['18-99']}


In [2]:
#Data for matching result index
DATAPATH = 'data/final_transformed_data.csv' 

df_original = pd.read_csv(DATAPATH)
#df = df[~df.overview.isna()]
print(len(df_original))
print(df_original.info())
print(df_original.head(5))

13344
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13344 entries, 0 to 13343
Data columns (total 86 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   supplier_sample_pulls__created_at               13344 non-null  int64  
 1   supplier_sample_pulls__sampling_method          13344 non-null  object 
 2   sample_pulls__target_groups                     13344 non-null  object 
 3   sample_pulls__country                           13344 non-null  object 
 4   sample_pulls__language                          13344 non-null  object 
 5   sample_pulls__cost_per_interview                13344 non-null  float64
 6   sample_pulls__incident_rate                     13344 non-null  float64
 7   sample_pulls__length_of_interview               13344 non-null  int64  
 8   sample_pulls__completes_needed                  13344 non-null  int64  
 9   sample_pulls__start_date         

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (80,85) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [108]:
df_original[['rl','continent_language_study_combine','projects__target_groups_qualifications_combine']].to_csv('sample_validation_reference_data.csv', index=False)

In [4]:
#filter wise list of sample projects to refer for requirement parameters for test inference.
#df = df_original.query("sample_pulls__start_date in ('2022-06-20','2022-06-21','2022-06-22')")
#df = df.query("sample_pulls__country != 'usa'")
#df = df.query("projects__study_types_ids in ('B2B','Consumer Study')")
df_inference_project = df_original.drop_duplicates(subset=['projects__id'])
#df_inference_project.info()
#df_inference_project[['sample_pulls__start_date','projects__name','sample_pulls__country','continent','sample_pulls__language','projects__study_types_ids','projects__study_types_subject_ids','sentence_1','sentence_2']].to_csv('dfs/unique_project.csv', index=False)
df_inference_project[['projects__name','projects__study_types_subject_ids','continent_language_study_combine','projects__target_groups_qualifications_combine']].to_csv('unique_project_new.csv', index=False)

In [5]:
df_inference_project.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2015 entries, 0 to 13329
Data columns (total 86 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   supplier_sample_pulls__created_at               2015 non-null   int64  
 1   supplier_sample_pulls__sampling_method          2015 non-null   object 
 2   sample_pulls__target_groups                     2015 non-null   object 
 3   sample_pulls__country                           2015 non-null   object 
 4   sample_pulls__language                          2015 non-null   object 
 5   sample_pulls__cost_per_interview                2015 non-null   float64
 6   sample_pulls__incident_rate                     2015 non-null   float64
 7   sample_pulls__length_of_interview               2015 non-null   int64  
 8   sample_pulls__completes_needed                  2015 non-null   int64  
 9   sample_pulls__start_date                

In [6]:
#STOPWORDS = set(stopwords.words('english'))
MIN_WORDS = 4
MAX_WORDS = 200

def tokenizer(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=None, lemmatize=False):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    
    tokens = [w for w in word_tokenize(sentence)]
    print(tokens)
    '''
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    '''
    return tokens
    
def extract_best_indices(m, topk, mask=None):
    """
    Use sum of the cosine distance over all tokens.
    m (np.array): cos matrix of shape (nb_in_tokens, nb_dict_tokens)
    topk (int): number of indices to return (from high to lowest in order)
    """
    global cos_sim
    cos_sim = []
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score 
    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:topk]

    return best_index

def get_recommendations_tfidf(vectorizer, sentence, tfidf_mat):
    
    """
    Return the database sentences in order of highest cosine similarity relatively to each 
    token of the target sentence. 
    """
    # Embed the query sentence
    tokens = [str(tok) for tok in tokenizer(sentence)]
    vec = vectorizer.transform(tokens)
    # Create list with similarity between query and dataset
    mat = cosine_similarity(vec, tfidf_mat)
    # Best cosine distance for each token independantly
    print(mat.shape)
    best_index = extract_best_indices(mat, topk=50)
    return best_index

#Load saved model tfidf embedding
def load_tfidf_embedding(sentence):
    with open(f'model/tfidf_model_{sentence}.pkl', 'rb') as fn:
        model = pk.load(fn)
    return model

#Load saved model tfidf vectorizer
def load_tfidf_vectorizer(vectorizer):
    with open(f'model/tfidf_{vectorizer}.pkl', 'rb') as fn:
        model = pk.load(fn)
    return model

In [18]:
'''
import QM_IR_train_script
with open(f'model/tfidf_vectorizer_1.pkl', 'rb') as fn:
        model = pk.load(fn)
'''

In [7]:
#import QM_IR_train_script
tfidf_embedding_1 = load_tfidf_embedding('sentence_1')
tfidf_vectorizer_1 = load_tfidf_vectorizer('vectorizer_1')
tfidf_embedding_2 = load_tfidf_embedding('sentence_2')
tfidf_vectorizer_2 = load_tfidf_vectorizer('vectorizer_2')

In [105]:
#query_sentence_1 = req_continent_language_study
query_sentence_1 = 'North America eng B2B'
query_sentence_2 = "{'GENDER': ['Female', 'Male']}"

best_tfidf_index_1 = get_recommendations_tfidf(tfidf_vectorizer_1, query_sentence_1, tfidf_embedding_1)
#get matched supplier rl and associate score
matched_df_1 = df_original[['rl']].iloc[best_tfidf_index_1]

best_cos_sim_list = []
total_cos_sim_list = cos_sim.tolist()

for best_index in best_tfidf_index_1.tolist():
    best_cos_sim_list.extend([total_cos_sim_list[best_index]])

rl_score_dict_1 = {}
i = 0

#create rl & match score dict
for rl in matched_df_1['rl'].tolist():
    if rl in rl_score_dict_1.keys():
        if rl_score_dict_1[rl] < best_cos_sim_list[i]:
            rl_score_dict_1[rl] = best_cos_sim_list[i]
    else:
        rl_score_dict_1[rl] = best_cos_sim_list[i]
    i+=1
print(rl_score_dict_1)
print(set(matched_df_1['rl'].tolist()))
#display(matched_df_1)

['North', 'America', 'eng', 'B2B']
['north']
['america']
['eng']
['b2b']
(4, 13344)
{537: 0.4825369053380314, 601: 0.4825369053380314, 600: 0.4825369053380314, 271: 0.4825369053380314, 500: 0.4825369053380314, 448: 0.4825369053380314, 273: 0.4825369053380314, 293: 0.4825369053380314, 558: 0.4825369053380314}
{448, 293, 537, 271, 558, 273, 500, 600, 601}


In [106]:
#Calculate match-prediction for sentence_2, for given req parameters
#query_sentence_2 = "{'AGE': ['18-99']}"
#query_sentence_2 = req_tg_qualification

best_tfidf_index_2 = get_recommendations_tfidf(tfidf_vectorizer_2, query_sentence_2, tfidf_embedding_2)
matched_df_2 = df_original[['rl']].iloc[best_tfidf_index_2]

#get matched supplier rl and associate score
best_cos_sim_list = []
total_cos_sim_list = cos_sim.tolist()

for best_index in best_tfidf_index_2.tolist():
    best_cos_sim_list.extend([total_cos_sim_list[best_index]])

rl_score_dict_2 = {}
i = 0

#create rl & match score dict
for rl in matched_df_2['rl'].tolist():
    if rl in rl_score_dict_2.keys():
        if rl_score_dict_2[rl] < best_cos_sim_list[i]:
            rl_score_dict_2[rl] = best_cos_sim_list[i]
    else:
        rl_score_dict_2[rl] = best_cos_sim_list[i]
    i+=1
print(rl_score_dict_2)
#display(matched_df_2)

['{', "'GENDER", "'", ':', '[', "'Female", "'", ',', "'Male", "'", ']', '}']
['{']
["'gender"]
["'"]
[':']
['[']
["'female"]
["'"]
[',']
["'male"]
["'"]
[']']
['}']
(12, 13344)
{588: 0.32591241332683313, 500: 0.32591241332683313, 359: 0.32591241332683313, 601: 0.32591241332683313, 273: 0.32591241332683313, 600: 0.32591241332683313, 537: 0.32591241332683313, 271: 0.32591241332683313, 593: 0.32591241332683313, 448: 0.32591241332683313, 401: 0.32591241332683313, 562: 0.32591241332683313, 124: 0.32591241332683313, 301: 0.32591241332683313, 574: 0.32591241332683313, 565: 0.32591241332683313}


In [107]:
#find common supplier keys
common_suppier_rl = [x for x in rl_score_dict_1 if x in rl_score_dict_2]
print(common_suppier_rl)

[537, 601, 600, 271, 500, 448, 273]


In [16]:
#set scaling factor w.r.t 100%
scale_factor_1 = 1.93
high_score_1 = 0.35*scale_factor_1
medium_score_1 = 0.16*scale_factor_1

scale_factor_2 = 2.75
high_score_2 = 0.25*scale_factor_2
medium_score_2 = 0.11*scale_factor_2


In [ ]:
'''
#set scaling factor w.r.t 100%
scale_factor_1 = 1.5
high_score_1 = 0.45*scale_factor_1
medium_score_1 = 0.2*scale_factor_1

scale_factor_2 = 2.1
high_score_2 = 0.32*scale_factor_2
medium_score_2 = 0.14*scale_factor_2
'''

In [24]:
import statistics
common_suppier_rl_high = {}
common_suppier_rl_medium_high = {}
common_suppier_rl_high_medium = {}
common_suppier_rl_medium_medium = {}
common_suppier_rl_low_high = {}
common_suppier_rl_high_low = {}
threshold = float(0.30)

for rl_item in common_suppier_rl:
    #print((rl_score_dict_1[rl_item])*scale_factor_1)
    #high & high
    if (float(rl_score_dict_1[rl_item])*scale_factor_1 > high_score_1) and (float(rl_score_dict_2[rl_item])*scale_factor_2 > high_score_2):
        rl_mean = statistics.mean([float(rl_score_dict_1[rl_item]*scale_factor_1),float(rl_score_dict_2[rl_item]*scale_factor_2)])
        
        if rl_mean > threshold:
            common_suppier_rl_high[rl_item] = round(rl_mean,4)

    #medium & high
    elif (float(rl_score_dict_1[rl_item])*scale_factor_1 < high_score_1) and (float(rl_score_dict_1[rl_item])*scale_factor_1 > medium_score_1) and (float(rl_score_dict_2[rl_item])*scale_factor_2 > high_score_2):
        rl_mean = statistics.mean([float(rl_score_dict_1[rl_item]*scale_factor_1),float(rl_score_dict_2[rl_item]*scale_factor_2)])
        
        if rl_mean > threshold:
            common_suppier_rl_medium_high[rl_item] = round(rl_mean,4)

    #high & medium
    elif (float(rl_score_dict_1[rl_item])*scale_factor_1 > high_score_1) and (float(rl_score_dict_2[rl_item])*scale_factor_2 < high_score_2) and (float(rl_score_dict_2[rl_item])*scale_factor_2 > medium_score_2):
        rl_mean = statistics.mean([float(rl_score_dict_1[rl_item]*scale_factor_1),float(rl_score_dict_2[rl_item]*scale_factor_2)])
        
        if rl_mean > threshold:
            common_suppier_rl_high_medium[rl_item] = round(rl_mean,4)

    #medium & medium
    elif (float(rl_score_dict_1[rl_item])*scale_factor_1 < high_score_1) and (float(rl_score_dict_1[rl_item])*scale_factor_1 > medium_score_1) and (float(rl_score_dict_2[rl_item])*scale_factor_2 < high_score_2) and (float(rl_score_dict_2[rl_item])*scale_factor_2 > medium_score_2):
        rl_mean = statistics.mean([float(rl_score_dict_1[rl_item]*scale_factor_1),float(rl_score_dict_2[rl_item]*scale_factor_2)])
        
        if rl_mean > threshold:
            common_suppier_rl_medium_medium[rl_item] = round(rl_mean,4)

    #low & high       
    elif (float(rl_score_dict_1[rl_item])*scale_factor_1 < medium_score_1) and (float(rl_score_dict_2[rl_item])*scale_factor_2 > high_score_2):
        rl_mean = statistics.mean([float(rl_score_dict_1[rl_item]*scale_factor_1),float(rl_score_dict_2[rl_item]*scale_factor_2)])
        
        if rl_mean > threshold:
            common_suppier_rl_low_high[rl_item] = round(rl_mean,4)

    #high & low
    elif (float(rl_score_dict_1[rl_item])*scale_factor_1 > high_score_1) and (float(rl_score_dict_2[rl_item])*scale_factor_2 < medium_score_2):
        rl_mean = statistics.mean([float(rl_score_dict_1[rl_item]*scale_factor_1),float(rl_score_dict_2[rl_item]*scale_factor_2)])
        
        if rl_mean > threshold:
            common_suppier_rl_high_low[rl_item] = round(rl_mean,4)

common_suppier_rl_high = dict(sorted(common_suppier_rl_high.items(), key=lambda item: item[1], reverse=True))
common_suppier_rl_medium_high = dict(sorted(common_suppier_rl_medium_high.items(), key=lambda item: item[1], reverse=True))
common_suppier_rl_high_medium = dict(sorted(common_suppier_rl_high_medium.items(), key=lambda item: item[1], reverse=True))
common_suppier_rl_medium_medium = dict(sorted(common_suppier_rl_medium_medium.items(), key=lambda item: item[1], reverse=True))
common_suppier_rl_low_high = dict(sorted(common_suppier_rl_low_high.items(), key=lambda item: item[1], reverse=True))
common_suppier_rl_high_low = dict(sorted(common_suppier_rl_high_low.items(), key=lambda item: item[1], reverse=True))

final_supplier_score_dict = common_suppier_rl_high.copy()
final_supplier_score_dict |= common_suppier_rl_medium_high
final_supplier_score_dict |= common_suppier_rl_high_medium
final_supplier_score_dict |= common_suppier_rl_medium_medium
final_supplier_score_dict |= common_suppier_rl_low_high
final_supplier_score_dict |= common_suppier_rl_high_low

print(final_supplier_score_dict)


{537: 0.9631, 601: 0.9631, 600: 0.9631, 271: 0.9631, 500: 0.9631, 273: 0.9631, 558: 0.9631}


In [ ]:
{601: 0.7689118340518992, 271: 0.7689118340518992, 537: 0.7689118340518992, 113: 0.7689118340518992, 600: 0.7689118340518992, 301: 0.7689118340518992, 574: 0.7689118340518992, 557: 0.7689118340518992, 273: 0.663358929986784, 500: 0.663358929986784}

In [ ]:
{601: 0.9980158101917733, 271: 0.9980158101917733, 537: 0.9980158101917733, 113: 0.9980158101917733, 600: 0.9980158101917733, 301: 0.9980158101917733, 574: 0.9980158101917733, 557: 0.9980158101917733, 273: 0.8622044069613252, 500: 0.8622044069613252}

In [ ]:
{601: 0.4402530251040739, 271: 0.4402530251040739, 537: 0.4402530251040739, 113: 0.4402530251040739, 600: 0.4402530251040739, 301: 0.4402530251040739, 574: 0.4402530251040739, 557: 0.4402530251040739, 273: 0.3698844223939971, 500: 0.3698844223939971}

In [ ]:
#combining 1 & 2 list and dropping duplicate supplier rl values.
rl_list = []
rl_list = matched_df_1['rl'].tolist()
rl_list.extend(matched_df_2['rl'].tolist())
rl_list_unique = list(set(rl_list))

In [ ]:
#prepare parameters to be sent to next recommendation pipeline.
request_param_dict = {}
request_param_dict['supplier_rl'] = rl_list_unique
request_param_dict['req_study_type'] = req_study_type
request_param_dict['req_subject'] = req_subject
request_param_dict['req_lang'] = req_lang
request_param_dict['req_country'] = req_country
request_param_dict['req_continent'] = req_continent

In [ ]:
print(rl_list)
print(rl_list_unique)
print(request_param_dict)

In [ ]:
#Calculate match-prediction for sentence_2
#query_sentence_2 = "{'GENDER': ['Female'], 'AGE': ['30-40']}"
query_sentence_2 = "{'AGE': ['18-34,45-65']}"
#query_sentence_2 = "{'AGE': ['25-65']}"
#query_sentence_2 = "{'GENDER': ['Female', 'Male'], 'AGE': ['18-54']}"
#query_sentence_2 = "{'GENDER': ['Female', 'Male'], 'AGE': ['18-30']}"
#query_sentence_2 = "STANDARD_COMPANY_DEPARTMENT': ['Technology Implementation', 'Finance\\/Accounting', 'Operations', 'Procurement'], 'STANDARD_EMPLOYMENT': ['Self-employed full-time', 'Employed part-time', 'Employed full-time'], 'GENDER': ['Female', 'Male'], 'AGE': ['18-99']}"
#query_sentence_2 = "{'STANDARD_INDUSTRY_PERSONAL': ['Carpenting\\/Electrical installations\\/VVS', 'Construction'], 'STANDARD_EMPLOYMENT': ['Self-employed part-time', 'Self-employed full-time', 'Employed part-time', 'Employed full-time'], 'GENDER': ['Female', 'Male'], 'AGE': ['18-70']}"
#query_sentence_2 = "{'STANDARD_HHI': ['$200,000 to $249,999', '$150,000 to $174,999', '$55,000 to $59,999', '$125,000 to $149,999', '$250,000 and above', '$100,000 to $124,999', '$95,000 to $99,999', '$70,000 to $74,999', '$85,000 to $89,999', '$80,000 to $84,999', '$65,000 to $69,999', '$40,000 to $44,999', '$45,000 to $49,999', '$75,000 to $79,999', '$90,000 to $94,999', '$50,000 to $54,999', '$175,000 to $199,999', '$35,000 to $39,999', '$30,000 to $34,999', '$60,000 to $64,999'], 'AGE': ['18-99']}"
#query_sentence_2 = "{'DMA': ['GREENVILLE-SPARTA-ASHEVILLE | 567', 'PARKERSBURG | 597', 'COLUMBIA, SC | 546', 'CHARLOTTESVILLE | 584', 'GREENWOOD-GREENVILLE | 647', 'PHOENIX-PRESCOTT | 753'], 'GENDER': ['Female', 'Male'], 'AGE': ['18-65']}"
#query_sentence_2 = "{'STANDARD_B2B_DECISION_MAKER': ['Other', 'Shipping', 'IT Software', 'Sales', 'Legal services', 'Marketing\\/Advertising', 'Office supplies', 'Auto leasing\\/purchasing', 'Food services', 'Corporate travel', 'Human Resources', 'Financial Department', 'IT Hardware', 'Telecommunications', 'Security', 'Operations', 'Printers and copiers'], 'STANDARD_JOB_TITLE': ['Manager (Group Manager, Sr. Manager, Manager, Program Manager)', 'Director (Group Director, Sr. Director, Director)', 'C-Level (e.g. CEO, CFO), Owner, Partner, President', 'Vice President (EVP, SVP, AVP, VP)'], 'STANDARD_EMPLOYMENT': ['Self-employed full-time', 'Employed part-time', 'Employed full-time'], 'GENDER': ['Female', 'Male'], 'AGE': ['25-99']}"
#query_sentence_2 = "{'STANDARD_PRIMARY_DECISION_MAKER': ['Yes', 'Share decisions equally'], 'AGE': ['18-99']}"
best_tfdif_index_2 = get_recommendations_tfidf(tfidf_vectorizer_2, query_sentence_2, tfidf_embedding_2)
df_suggestion_list = df_original[['rl','sentence_2']].iloc[best_tfdif_index_2]
df_suggestion_list = df_suggestion_list.drop_duplicates(subset=['rl'])
df_suggest_final = df_suggestion_list[['rl']]
df_suggest_final['projects__name'] = 'parents of children'
df_suggest_final['req_study_type'] = 'Consumer Study'
df_suggest_final['req_subject'] = 'other'
df_suggest_final['req_lang'] = 'eng'
df_suggest_final['req_country'] = 'gbr'
df_suggest_final['req_continent'] = 'Europe'
df_suggest_final.to_csv('dfs/suggested_suppliers.csv', index=False)
